In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
dados_us = pd.read_excel("Base Cotacoes.xlsx",sheet_name='Dados US',index_col=0)

dados_brl = pd.read_excel("Base Cotacoes.xlsx",sheet_name='Dados BR Cupom Limpo',index_col=0)

dolar = pd.read_excel("Dolar Cupom Limpo.xlsx",index_col=0)

anos = list(dados_brl.index.year.unique())

retornos_brl = dados_brl.pct_change().dropna()

retornos_us = dados_us.pct_change().dropna()

In [3]:
retornos_anuais_us = pd.DataFrame()
retornos_anuais_br = pd.DataFrame()
retornos_dolar = pd.DataFrame()

for ano in anos:
    dados_corte = dados_us.loc[dados_us.index.year == ano]

    ret_anual = (dados_corte/dados_corte.iloc[0])-1
    ret_anual = ret_anual.iloc[-1]
    ret_anual = pd.DataFrame(ret_anual).T
    retornos_anuais_us = pd.concat([retornos_anuais_us,ret_anual])

for ano in anos:
    dados_corte = dados_brl.loc[dados_brl.index.year == ano]

    ret_anual = (dados_corte/dados_corte.iloc[0])-1
    ret_anual = ret_anual.iloc[-1]
    ret_anual = pd.DataFrame(ret_anual).T
    retornos_anuais_br = pd.concat([retornos_anuais_br,ret_anual])

for ano in anos:
    dados_corte = dados_us.loc[dados_us.index.year == ano]

    ret_anual = (dados_corte/dados_corte.iloc[0])-1
    ret_anual = ret_anual.iloc[-1]
    ret_anual = pd.DataFrame(ret_anual).T
    retornos_anuais_us = pd.concat([retornos_anuais_us,ret_anual])



In [4]:
retornos_anuais_br.index = retornos_anuais_br.index.year
retornos_anuais_us.index = retornos_anuais_us.index.year

In [5]:
media_5anos_br = retornos_anuais_br.rolling(5).mean()
media_5anos_us = retornos_anuais_us.rolling(5).mean()

In [6]:
visoes_absolutas_br = (media_5anos_br - retornos_anuais_br).dropna()
visoes_absolutas_us = (media_5anos_us - retornos_anuais_us).dropna()

In [7]:
########################################################################################################################################
# Criação das visões relativas em BRL
visoes_relativas_br = pd.DataFrame()


for coluna in visoes_absolutas_br.columns:
    
    subtracoes = visoes_absolutas_br.apply(lambda row: row[coluna] - row, axis=1)
    
   
    visoes_relativas_br = pd.concat([visoes_relativas_br, subtracoes], axis=1)


visoes_relativas_br.columns = [f'{coluna}_sub_{outra_coluna}' for coluna in visoes_absolutas_br.columns for outra_coluna in visoes_absolutas_br.columns]

visoes_relativas_br.dropna(inplace=True)

visoes_relativas_br.replace(0,np.nan,inplace=True)

visoes_relativas_br.dropna(axis=1,inplace=True)
########################################################################################################################################

########################################################################################################################################
# Criação das visões relativas em USD

visoes_relativas_us = pd.DataFrame()


for coluna in visoes_absolutas_us.columns:
    
    subtracoes = visoes_absolutas_us.apply(lambda row: row[coluna] - row, axis=1)
    
    
    visoes_relativas_us = pd.concat([visoes_relativas_us, subtracoes], axis=1)

visoes_relativas_us.columns = [f'{coluna}_sub_{outra_coluna2}' for coluna in visoes_absolutas_us.columns for outra_coluna2 in visoes_absolutas_us.columns]

visoes_relativas_us.dropna(inplace=True)

visoes_relativas_us.replace(0,np.nan,inplace=True)

visoes_relativas_us.dropna(axis=1,inplace=True)
########################################################################################################################################



In [2]:
from bcb import PTAX

PTAX()


# from bcb import Expectativas 
# Expectativas()
# em.describe()
# em.describe('ExpectativasMercadoTop5Anuais')
# ep = em.get_endpoint('ExpectativasMercadoAnuais')
# focus = ep.query().filter(ep.Indicador == 'Câmbio').collect()
# focus.Indicador.unique()


# expectativa_cambio_focus = focus.loc[(focus['Indicador'] == 'Câmbio') & (focus['Data'] >=str(anos[-1])+str('-01-01'))]

In [10]:
#########################################################################################
################## Filtros para BOLETIM FOCUS ###########################################
#  ExpectativasMercadoTop5Anuais
#  ExpectativaMercadoMensais
#  ExpectativasMercadoInflacao24Meses
#  ExpectativasMercadoInflacao12Meses
#  ExpectativasMercadoSelic
#  ExpectativasMercadoTop5Selic
#  ExpectativasMercadoTop5Mensais
#  ExpectativasMercadoTrimestrais
#  ExpectativasMercadoAnuais
#########################################################################################


#########################################################################################
################## Indicadores possíveis ################################################
#['Balança comercial', 'Conta corrente', 'Resultado primário',
#       'IPCA-15', 'PIB Agropecuária', 'PIB Indústria', 'PIB Serviços',
#       'PIB Total', 'IPCA Administrados', 'Investimento direto no país',
#       'Resultado nominal', 'Dívida líquida do setor público',
#       'Dívida bruta do governo geral', 'IGP-DI', 'IGP-M', 'INPC',
#       'IPA-DI', 'IPA-M', 'IPCA', 'IPC-Fipe', 'Produção industrial',
#      'Selic', 'Câmbio', 'Taxa de desocupação',
#       'PIB Despesa de consumo da administração pública',
#      'PIB Formação Bruta de Capital Fixo',
#       'PIB Exportação de bens e serviços',
#       'PIB Importação de bens e serviços', 'IPCA Livres',
#       'IPCA Serviços', 'IPCA Bens industrializados',
#       'IPCA Alimentação no domicílio',
#       'PIB Despesa de consumo das famílias']
#########################################################################################
#########################################################################################

# Se erro verificar se o site https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/documentacao está online

from bcb import Expectativas

em = Expectativas() 
em.describe()
ep = em.get_endpoint('ExpectativasMercadoAnuais')
focus = ep.query().filter(ep.Indicador == 'Câmbio').collect()
focus.Indicador.unique()


expectativa_cambio_focus = focus.loc[(focus['Indicador'] == 'Câmbio') & (focus['Data'] >=str(anos[-1])+str('-01-01'))]

EntitySets:
  ExpectativasMercadoTop5Anuais
  ExpectativaMercadoMensais
  ExpectativasMercadoInflacao24Meses
  ExpectativasMercadoInflacao12Meses
  ExpectativasMercadoSelic
  ExpectativasMercadoTop5Selic
  ExpectativasMercadoTop5Mensais
  ExpectativasMercadoTrimestrais
  ExpectativasMercadoAnuais


In [11]:
expectativa_cambio_focus

,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo
32377,Câmbio,None,2024-01-02,2024,4.8878,4.910,0.1928,4.50,5.2500,26.0,1
32378,Câmbio,None,2024-01-02,2025,4.9500,5.050,0.2427,4.40,5.3000,21.0,1
32379,Câmbio,None,2024-01-02,2026,4.9805,5.100,0.2813,4.40,5.3500,19.0,1
32380,Câmbio,None,2024-01-02,2027,5.0068,5.100,0.3341,4.40,5.5000,17.0,1
32381,Câmbio,None,2024-01-02,2028,4.9533,5.045,0.3812,4.40,5.3800,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...
33222,Câmbio,None,2024-05-03,2024,5.0239,5.000,0.1068,4.65,5.3400,125.0,0
33223,Câmbio,None,2024-05-03,2025,5.0558,5.050,0.1556,4.50,5.5000,122.0,0
33224,Câmbio,None,2024-05-03,2026,5.0725,5.100,0.1831,4.50,5.5000,92.0,0
33225,Câmbio,None,2024-05-03,2027,5.0932,5.100,0.2070,4.40,5.7316,86.0,0


In [ ]:
########### Captação da média do focus ao longo do ano corrente #####################################


focus_2024 = expectativa_cambio_focus.loc[expectativa_cambio_focus['DataReferencia'] == '2024']['Mediana'].mean()

focus_2025 = expectativa_cambio_focus.loc[expectativa_cambio_focus['DataReferencia'] == '2025']['Mediana'].mean()


print(focus_2024)

print(focus_2025)

(focus_2025/focus_2024)-1

4.940665909090908
5.002973484848484


0.012611169608317097

In [42]:
market_cap_us = pd.read_excel('Dados_MarketCap.xlsx',header=4,index_col=0).sort_index()


dolar['Ano'] = dolar.index.year
dolar_fechamento = dolar.groupby('Ano').tail(1)
dolar_fechamento = dolar_fechamento.loc[dolar_fechamento.index.isin(market_cap_us.index)]


market_cap_br = market_cap_us.mul(dolar_fechamento['Cupom Limpo'],axis=0)

In [44]:
dolar_fechamento

,Cupom Limpo,Ano
Data,,
2012-12-31,2.0435,2012
2013-12-31,2.3426,2013
2014-12-31,2.6562,2014
2015-12-31,3.9048,2015
2016-12-30,3.2591,2016
2017-12-29,3.3080,2017
2018-12-31,3.8748,2018
2019-12-31,4.0307,2019
2020-12-31,5.1967,2020


In [ ]:

# Escrever os DataFrames em um arquivo Excel com abas separadas
with pd.ExcelWriter('Matriz Base.xlsx', engine='openpyxl') as writer:
    dados_brl.to_excel(writer,sheet_name='Dados BR',index=True)
    dados_us.to_excel(writer,sheet_name='Dados US', index=True)
    retornos_anuais_us.to_excel(writer, sheet_name='Retornos Anuais US', index=True)
    retornos_anuais_br.to_excel(writer, sheet_name='Retornos Anuais BR', index=True)
    visoes_absolutas_us.to_excel(writer, sheet_name='Visoes Absolutas US', index=True)
    visoes_absolutas_br.to_excel(writer, sheet_name='Visoes Absolutas BR', index=True)
    visoes_relativas_us.to_excel(writer, sheet_name='Visoes Relativas US', index=True)
    visoes_relativas_br.to_excel(writer, sheet_name='Visoes Relativas BR', index=True)
    expectativa_cambio_focus.to_excel(writer, sheet_name= 'Boletim Focus Câmbio', index= True)
    market_cap_us.to_excel(writer,sheet_name='Market Cap US', index=True)
    market_cap_br.to_excel(writer,sheet_name='Market Cap BR', index=True)

